In [1]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl.create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
intents = [
    # Greetings
    {
        'tag': 'greeting',
        'patterns': ['hi', 'hello', 'hey', 'whats up', 'how are you'],
        'responses': ['Hi there!', 'Hello!', 'Hey!', 'Not much, what about you?', 'I’m doing great, thanks for asking!']
    },

    # Farewells
    {
        'tag': 'goodbye',
        'patterns': ['bye', 'see you', 'goodbye', 'take care', 'later'],
        'responses': ['Goodbye!', 'Take care!', 'See you later!', 'Have a great day!']
    },

    # Gratitude
    {
        'tag': 'thanks',
        'patterns': ['thanks', 'thank you', 'appreciate it', 'many thanks'],
        'responses': ['You’re welcome!', 'Happy to help!', 'No problem!', 'Anytime!']
    },

    # Apologies
    {
        'tag': 'apologies',
        'patterns': ['sorry', 'my apologies', 'pardon me', 'excuse me'],
        'responses': ['It’s okay!', 'No worries!', 'Don’t worry about it!', 'All good!']
    },

    # Help/Assistance
    {
        'tag': 'help',
        'patterns': ['help me', 'can you help', 'i need assistance', 'what can you do', 'help'],
        'responses': ['Of course! What do you need help with?', 'Sure, I’m here to assist you!', 'Happy to help! Let me know what you need.']
    },

    # Emotions: Happy
    {
        'tag': 'happy',
        'patterns': ['i’m happy', 'feeling great', 'i’m excited', 'so joyful'],
        'responses': ['That’s wonderful to hear!', 'Yay! I’m glad to hear that!', 'Awesome, let’s keep the good vibes going!']
    },

    # Emotions: Sad
    {
        'tag': 'sad',
        'patterns': ['i’m sad', 'feeling down', 'not good', 'i feel bad', 'i’m upset'],
        'responses': ['I’m here for you.', 'Want to talk about it?', 'I hope things get better soon.', 'I’m sorry to hear that.']
    },

    # Emotions: Angry
    {
        'tag': 'angry',
        'patterns': ['i’m angry', 'so mad', 'really frustrated', 'i’m upset'],
        'responses': ['I understand, do you want to share what’s upsetting you?', 'Take a deep breath, I’m here to listen.', 'It’s okay to feel that way.']
    },

    # Weather
    {
        'tag': 'weather',
        'patterns': ['what’s the weather', 'current weather', 'is it raining', 'temperature', 'weather today'],
        'responses': ['Let me check the weather for you.', 'It looks sunny today!', 'There might be rain, stay prepared!', 'It’s a bit chilly, dress warmly!']
    },

    # Jokes
    {
        'tag': 'jokes',
        'patterns': ['tell me a joke', 'make me laugh', 'say something funny', 'joke'],
        'responses': ['Why don’t scientists trust atoms? Because they make up everything!', 'Why was the math book sad? It had too many problems.', 'I would tell you a joke about a roof, but it’s over your head!']
    },

    # Stock Market
    {
        'tag': 'stock_market',
        'patterns': ['stock market news', 'tell me about stocks', 'any stock updates', 'how’s the market', 'stock trends'],
        'responses': ['Here’s the latest stock market update.', 'Let me fetch the stock trends for you.', 'The stock market is doing well today!', 'Check out these top-performing stocks!']
    },

    # Education
    {
        'tag': 'learn',
        'patterns': ['teach me', 'i want to learn', 'how do i', 'can you explain', 'help me understand'],
        'responses': ['Sure! What topic do you want to learn about?', 'Let me teach you step by step.', 'I’d be happy to explain it to you!', 'Let’s dive into it together.']
    },

    # Motivation
    {
        'tag': 'motivation',
        'patterns': ['i need motivation', 'inspire me', 'feeling lazy', 'help me stay motivated'],
        'responses': ['You’ve got this!', 'Believe in yourself!', 'Take one small step at a time.', 'Remember why you started!']
    },

    # Food
    {
        'tag': 'food',
        'patterns': ['i’m hungry', 'what should i eat', 'suggest food', 'recommend a dish'],
        'responses': ['How about some pasta?', 'Pizza always sounds good!', 'Why not try something healthy like a salad?', 'Craving anything specific?']
    },

    # Movies
    {
        'tag': 'movies',
        'patterns': ['recommend a movie', 'what to watch', 'suggest a film', 'movie ideas'],
        'responses': ['Have you watched Inception? It’s a classic!', 'How about a comedy to lighten the mood?', 'Looking for action, drama, or sci-fi?']
    },


    
    # Encouragement
    {
        'tag': 'encouragement',
        'patterns': ['i feel lost', 'i’m stuck', 'i need advice', 'help me out', 'can you guide me'],
        'responses': ['You’ve got this! Stay focused.', 'Take one step at a time, you’ll make it.', 'I believe in you.', 'Let’s figure this out together.']
    },

    # Travel
    {
        'tag': 'travel',
        'patterns': ['suggest a travel destination', 'where should I go', 'vacation ideas', 'i want to travel'],
        'responses': ['How about a beach getaway?', 'Try visiting a historical city.', 'Explore the mountains for a relaxing trip.', 'Looking for adventure or relaxation?']
    },

    # Fitness
    {
        'tag': 'fitness',
        'patterns': ['how to lose weight', 'fitness tips', 'exercise ideas', 'how to stay fit'],
        'responses': ['Consistency is key! Start small.', 'Try a mix of cardio and strength training.', 'Stay hydrated and eat balanced meals.', 'How about yoga for mindfulness and fitness?']
    },

    # Technology
    {
        'tag': 'technology',
        'patterns': ['latest technology', 'tech news', 'what’s new in tech', 'gadgets to buy'],
        'responses': ['Check out these trending gadgets!', 'AI is making waves in technology.', 'Want recommendations for gadgets?', 'Technology is evolving rapidly—what area are you interested in?']
    },

    # Life Advice
    {
        'tag': 'life_advice',
        'patterns': ['how to be happy', 'life tips', 'how to make decisions', 'advice on life'],
        'responses': ['Focus on what you can control.', 'Happiness comes from within.', 'Take time to reflect and prioritize.', 'Stay true to your values and goals.']
    },

    # Anxiety
    {
        'tag': 'anxiety',
        'patterns': ['i feel anxious', 'how to calm down', 'i’m nervous', 'how to deal with anxiety'],
        'responses': ['Take deep breaths, you’re doing great.', 'Try to focus on the present moment.', 'It’s okay to feel this way, and it will pass.', 'Talk to someone you trust about how you feel.']
    },

    # Career
    {
        'tag': 'career',
        'patterns': ['career advice', 'help with my job', 'job tips', 'how to get a promotion'],
        'responses': ['Focus on building your skills.', 'Network with people in your field.', 'Keep learning and improving.', 'Set clear career goals and work towards them.']
    },

    # Sports
    {
        'tag': 'sports',
        'patterns': ['latest sports news', 'who won the match', 'sports updates', 'favorite sport'],
        'responses': ['Here’s the latest sports update.', 'It was a thrilling game!', 'Do you play any sports?', 'Which team are you rooting for?']
    },

    # Health
    {
        'tag': 'health',
        'patterns': ['how to stay healthy', 'health tips', 'how to avoid illness', 'what to eat for health'],
        'responses': ['Drink plenty of water and eat nutritious meals.', 'Get enough sleep and stay active.', 'Regular checkups can help prevent issues.', 'Listen to your body and take care of it.']
    },

    # Fun Facts
    {
        'tag': 'fun_facts',
        'patterns': ['tell me a fact', 'interesting information', 'give me trivia', 'any fun facts?'],
        'responses': ['Did you know honey never spoils?', 'A group of flamingos is called a "flamboyance".', 'Octopuses have three hearts!', 'Bananas are berries, but strawberries are not!']
    },

    # Productivity
    {
        'tag': 'productivity',
        'patterns': ['how to be productive', 'productivity tips', 'i feel lazy', 'how to stay focused'],
        'responses': ['Break tasks into small steps.', 'Set clear priorities for the day.', 'Take regular breaks to avoid burnout.', 'Stay organized with a planner or to-do list.']
    },

    # Fashion
    {
        'tag': 'fashion',
        'patterns': ['what to wear', 'latest fashion trends', 'style tips', 'how to look good'],
        'responses': ['Go for outfits that match your personality.', 'Layering is always a great option.', 'Neutral tones are trending right now.', 'What’s your favorite style?']
    },

    # Relationships
    {
        'tag': 'relationships',
        'patterns': ['relationship advice', 'how to fix a fight', 'i feel lonely', 'how to make friends'],
        'responses': ['Communication is key in any relationship.', 'Apologize sincerely and talk things out.', 'Focus on building trust and respect.', 'Be open to meeting new people.']
    },

    # Hobbies
    {
        'tag': 'hobbies',
        'patterns': ['suggest a hobby', 'what should I do for fun', 'new hobby ideas', 'how to spend my free time'],
        'responses': ['How about painting or drawing?', 'Learn to play an instrument.', 'Gardening can be relaxing and rewarding.', 'Why not try hiking or exploring nature?']
    },

    # Dreams
    {
        'tag': 'dreams',
        'patterns': ['what do dreams mean', 'i had a weird dream', 'why do i dream', 'interpret my dream'],
        'responses': ['Dreams often reflect your thoughts and emotions.', 'They can be your subconscious working things out.', 'Want to share your dream? Maybe I can help interpret it!', 'Dreams can be fascinating—sometimes they’re just random though.']
    },

    # General Knowledge
    {
        'tag': 'general_knowledge',
        'patterns': ['give me general knowledge', 'teach me something new', 'interesting topics to learn', 'what’s something cool?'],
        'responses': ['The Great Wall of China can’t be seen from space—it’s a myth!', 'Sharks existed before trees!', 'The Eiffel Tower can grow taller in the summer due to heat expansion.', 'Did you know the first computer was invented in the 1940s?']
    },

    # Shopping
    {
        'tag': 'shopping',
        'patterns': ['shopping tips', 'where to buy', 'best shopping sites', 'recommend a store'],
        'responses': ['Online shopping platforms like Amazon or Flipkart have a lot of options.', 'Look for deals and discounts before making a purchase.', 'What are you looking to buy?', 'Local markets can have unique finds!']
    },

    # Music
    {
        'tag': 'music',
        'patterns': ['recommend a song', 'what’s trending in music', 'favorite songs', 'new music to listen to'],
        'responses': ['Have you heard the latest hits?', 'Here are some trending songs you might like.', 'What’s your favorite genre of music?', 'How about creating a playlist together?']
    }
]